# Notes
These are just notes I used for constructing the TFP joints

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability.python.internal import tensorshape_util
tfp.__version__

'0.11.0'

In [2]:
import maxentep

In [3]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
print(tf.where(bs > 0)[:,1:])


tf.Tensor(
[[0 1]
 [1 2]
 [2 3]
 [3 3]
 [0 1]
 [1 2]
 [2 3]
 [3 3]], shape=(8, 2), dtype=int64)


In [4]:
j = maxentep.recip_norm_mat_dist(start, var)
print(j)
j.sample()

tfp.distributions.JointDistributionSequentialAutoBatched("JointDistributionSequentialAutoBatched", batch_shape=[], event_shape=[[4], [4], [4, 4]], dtype=[float64, float64, float64])


[<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.14462493, 0.22296636, 0.56788011, 0.74673479])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.85537507, 0.77703364, 0.43211989, 0.25326521])>,
 <tf.Tensor: shape=(4, 4), dtype=float64, numpy=
 array([[0.85537507, 0.14462493, 0.        , 0.        ],
        [0.        , 0.77703364, 0.22296636, 0.        ],
        [0.        , 0.        , 0.43211989, 0.56788011],
        [0.        , 0.        , 0.        , 1.        ]])>]

In [5]:
j.log_prob(j.sample())

<tf.Tensor: shape=(), dtype=float64, numpy=4.259670892812701>

In [6]:
x = j.log_prob(j.sample(5))

In [ ]:
print(j.sample(4))
#j.log_prob(j.sample())

In [ ]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]]).astype(np.float32)
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
i = tf.reshape(tf.cast(tf.where(bs > 0)[...,1:], tf.int32), (-1, start.shape[0], 2))
v = tf.gather_nd(bs, i, batch_dims=1)
print(i, v)
b = maxentep.Scatter(i, bs.shape[1:])
bs.shape

In [ ]:
b.forward(v)
print(v, b.inverse(b.forward(v)))

In [ ]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]]).astype(np.float32)
i = tf.cast(tf.where(bs > 0)[...,1:], tf.int32)
v = tf.gather_nd(start, i)
print(i, v)
b = maxentep.Scatter(i,start.shape)
start.shape

In [ ]:
b.forward(v)
print(v, b.inverse(b.forward(v)))

In [ ]:
b.inverse_log_det_jacobian(v, 1)

In [ ]:
print(i)
tf.scatter_nd(i, v, (4,4))

In [ ]:
w = tf.constant(start.astype(np.float32))
w, tf.ones(tf.rank(w))

In [ ]:
i =  tf.keras.Input((1,))
combined = np.stack((start, var))
print(combined.shape)
x  = maxentep.TrainableInputLayer(combined, constraint=maxentep.PositiveMaskedConstraint(combined > 0))(i)

In [ ]:
m = tf.keras.Model(inputs=i, outputs=x)
t = m(tf.constant([1., 1.]))
t

In [ ]:
t[0,0], t[0,1]

In [ ]:
maxentep.recip_norm_mat_dist(t[:1,0], t[:1,1]).sample(5)

In [ ]:
i =  tf.keras.Input((1,))
l = maxentep.recip_norm_mat_layer(i, start, var, 'test')

In [ ]:
m = tf.keras.Model(inputs=i, outputs=l)

In [ ]:
m(tf.constant([1.]))